In [1]:
import os
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")
import tensorflow as tf
print(tf.test.gpu_device_name());
print("CPU count : ",os.cpu_count());

/device:GPU:0
CPU count :  12


#### Preparing dataset

importing required libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')
import librosa
import csv

training dataset
each audio file is classified as covid/not_covid

In [3]:
train_csv = pd.read_csv('Kaggle dataset\cough_trial_extended.csv')
train_csv

,file_properties,class
0,0v8MGxNetjg_ 10.000_ 20.000.wav,not_covid
1,1j1duoxdxBg_ 70.000_ 80.000.wav,not_covid
2,1MSYO4wgiag_ 120.000_ 130.000.wav,not_covid
3,1PajbAKd8Kg_ 0.000_ 10.000.wav,not_covid
4,cov1.wav,covid
...,...,...
165,-bZrDCS8KAg_ 70.000_ 80.000.wav,not_covid
166,-ej81N6Aqo4_ 0.000_ 8.000.wav,not_covid
167,-gvLnl1smfs_ 90.000_ 100.000.wav,not_covid
168,-hu5q-Nn4BM_ 70.000_ 80.000.wav,not_covid


In [7]:
train_csv['class'].unique()
tot_rows = train_csv.shape[0]

Creating spectrograms for each audio file

In [14]:
cmap = plt.get_cmap('inferno')
for i in range(tot_rows) :
    source = train_csv['file_properties'][i] # name of the audio file
    filename = 'Kaggle dataset/trial_covid/'+source
    y,sr = librosa.load(filename, mono=True, duration=5)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
    plt.axis('off')
    plt.savefig(f'spectrograms/{source[:-3].replace(".", "")}.png')
    plt.clf()

<Figure size 432x288 with 0 Axes>


Extracting features from Spectrogram¶

Features :

    Mel-frequency cepstral coefficients (MFCC)(20 in number)
    Spectral Centroid,
    Zero Crossing Rate
    Chroma Frequencies
    Spectral Roll-off.

In [8]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()
header

['chroma_stft',
 'rmse',
 'spectral_centroid',
 'spectral_bandwidth',
 'rolloff',
 'zero_crossing_rate',
 'mfcc1',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc14',
 'mfcc15',
 'mfcc16',
 'mfcc17',
 'mfcc18',
 'mfcc19',
 'mfcc20',
 'label']

writing to csv file

In [9]:
file = open('data_new_extended.csv', 'w')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for i in range(tot_rows):
    print("row ",i)
    source = train_csv['file_properties'][i]
    file_name = 'Kaggle dataset/trial_covid/'+source
    y,sr = librosa.load(file_name, mono=True, duration=5)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    to_append += ' '+train_csv['class'][i]
    to_append = to_append.split()
    file = open('data_new_extended.csv', 'a')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append)
    

row  0
row  1
row  2
row  3
row  4
row  5
row  6
row  7
row  8
row  9
row  10
row  11
row  12
row  13
row  14
row  15
row  16
row  17
row  18
row  19
row  20
row  21
row  22
row  23
row  24
row  25
row  26
row  27
row  28
row  29
row  30
row  31
row  32
row  33
row  34
row  35
row  36
row  37
row  38
row  39
row  40
row  41
row  42
row  43
row  44
row  45
row  46
row  47
row  48
row  49
row  50
row  51
row  52
row  53
row  54
row  55
row  56
row  57
row  58
row  59
row  60
row  61
row  62
row  63
row  64
row  65
row  66
row  67
row  68
row  69
row  70
row  71
row  72
row  73
row  74
row  75
row  76
row  77
row  78
row  79
row  80
row  81
row  82
row  83
row  84
row  85
row  86
row  87
row  88
row  89
row  90
row  91
row  92
row  93
row  94
row  95
row  96
row  97
row  98
row  99
row  100
row  101
row  102
row  103
row  104
row  105
row  106
row  107
row  108
row  109
row  110
row  111
row  112
row  113
row  114
row  115
row  116
row  117
row  118
row  119
row  120
row  121
row  122
row

In [10]:
data = pd.read_csv('data_new_extended.csv')
data

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.520202,0.045849,1613.207268,1412.419947,2908.029175,0.107019,-377.008667,110.928284,-31.870615,6.640702,...,-7.442677,-1.042071,-0.211678,-3.513029,-1.755283,-3.029743,-2.873752,-2.084186,-4.006258,not_covid
1,0.535920,0.001767,2892.900310,2467.700044,5074.209595,0.148584,-519.298096,60.833210,-13.686949,52.106445,...,-0.897269,7.246154,-1.711321,3.879874,3.637301,3.026381,2.439105,2.778507,2.209200,not_covid
2,0.495701,0.033442,3429.345493,2788.624737,6887.833659,0.225315,-282.546631,48.550930,-15.530038,12.706840,...,-6.072834,-4.181358,0.995803,-0.526877,0.544948,-8.846842,-2.909317,-1.115006,-5.921741,not_covid
3,0.407945,0.013446,2710.614836,2664.172992,5778.325399,0.142076,-346.925842,75.774315,-7.640137,11.369929,...,5.065131,-0.284327,0.986260,-2.453420,3.691722,2.317096,-2.061888,-4.779807,-0.513034,not_covid
4,0.413932,0.059003,1545.106644,1403.070020,2853.684400,0.130244,-340.659760,104.258057,-32.329517,-13.514756,...,-8.156581,0.851374,-5.614543,-6.410962,-0.998022,-1.892213,-2.441887,-3.758770,-3.492306,covid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0.494002,0.005091,1601.517754,2301.426512,3662.638346,0.047815,-543.939331,119.115051,23.888491,23.672983,...,-1.247695,-2.640601,1.630950,0.660486,1.616748,-2.473058,0.665920,-5.397944,-1.299975,not_covid
166,0.400460,0.052107,2664.429265,2563.518673,5518.431600,0.121514,-290.961487,85.464661,-10.990861,10.985048,...,-4.628566,-7.416304,-4.506893,-7.480810,-4.860178,-6.334512,-5.011344,-8.423484,-0.845706,not_covid
167,0.704503,0.058540,3090.063144,2740.902776,6531.090291,0.179077,-75.817581,68.875961,-11.661060,17.179529,...,0.688080,-7.519141,0.608991,-6.631441,0.658447,-4.130286,0.720673,-4.663277,-0.649811,not_covid
168,0.424427,0.044151,3174.170083,2482.889460,5768.206787,0.221743,-264.292847,58.730362,-7.444155,15.067248,...,-0.634892,0.677984,-3.650386,-6.033737,1.209816,-14.924580,-11.188300,-8.857522,-5.092669,not_covid


In [11]:
data.shape

(170, 27)

Encoding labels

1 is not_covid
0 is covid

In [12]:
genre_list = data.iloc[:,-1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

Scaling features

all columns except the last

In [14]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1],dtype=float))

train test split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [21]:
print(X_train.shape)
print(y_train.shape)

(113, 26)
(113,)


#### Classification Model using Tensorflow